In [ ]:
import csv
from datetime import datetime, timedelta
from time import sleep
import pandas as pd
import matplotlib.pyplot as plt
from stockstats import StockDataFrame 
import numpy as np
import pysentiment as ps
from collections import OrderedDict
import re
from os import listdir
hiv4 = ps.HIV4()

INITIAL_WEIGHT = 1;



indexesFiles=list()
newsFiles=list()
files=list()
newsFiles=listdir(r"C:\Users\Utente\Desktop\NLFF\NLFF\intrinioDataset")
indexesFiles=listdir(r"C:\Users\Utente\Desktop\NLFF\NLFF\DataSetIndexes")
#print(newsFiles)
#print(indexesFiles)
tickers=list()
for file in indexesFiles:
    file=re.sub('\.csv$', '', file)
    file=re.sub('indexes', '', file)
    tickers.append(file)
print(tickers)

In [ ]:
#tickers=["PCLN"]

for ticker in tickers:
    data = list()
    sentiment20minutes = list()
    timeSpan = list()
    print("Working on "+ticker+"...")
    
    #VARIABILE PER SETTARE DELAY DELLE NEWS IN MIN
    delay=20
    reader=pd.read_csv(r'C:\Users\Utente\Desktop\NLFF\NLFF\intrinioDataset\\'+ticker+'.csv') #reader = csv.DictReader(csvfile)
    for row in reader.T.iteritems():
        time  = datetime.strptime(row[1]['PUBLICATION_DATE'], '%Y-%m-%d %H:%M:%S +%f')
        print("first  "+time.strftime('%Y-%m-%d %H:%M:%S +%f'))
        time=time+timedelta(minutes=delay)
        print("then "+time.strftime('%Y-%m-%d %H:%M:%S +%f'))
        data.append({'time':time, 'summary':row[1]['SUMMARY']}) 

    #with open('indexesFB.csv', 'rb') as csvfile:
        #from time import sleep
        #sleep(1)
    reader = pd.read_csv(r'C:\Users\Utente\Desktop\NLFF\NLFF\DataSetIndexes\indexes'+ticker+'.csv')
    #print(reader.head())
    reader.rename(columns={'Unnamed: 0':'date'}, inplace=True)
    #print(reader.head())
    for row in reader.T.iteritems():
        time  = datetime.strptime(row[1]['date'], '%Y-%m-%d %H:%M:%S')
        timeSpan.append(time) 



    i = len(data)-1;
    j=0;
    
    
    initDate = max(timeSpan[0], data[len(data)-1]['time'])
    finalDate = min(timeSpan[len(timeSpan)-1], data[0]['time'])
    
     # ALLINEAMENTO INIZIO
    while(timeSpan[j] < initDate):
        j+=1
    while(data[i]['time'] <= initDate):
        i-=1
        
   ## while(timeSpan[j]< data[i]['time']):
        #j+=1

    weighted_sum = 0;
    normal_sum = 0
    while( data[i]['time'] < finalDate and timeSpan[j] < finalDate ):
    #while(i>0 and j<len(timeSpan)-1):
        
        total_subj = INITIAL_WEIGHT
        num_sentiment = 1
        
        initTime = timeSpan[j]
        
        

        # if(i == len(data)-1):
        # 	delta =timedelta(minutes=data[i]['time'].minute % 20, seconds=data[i]['time'].second, microseconds=data[i]['time'].microsecond)
        # 	initTime = data[i]['time'] - delta
        # else:
        # 	initTime = initTime + timedelta(minutes=20)

        # if(initTime.hour >= 22 and initTime.minute>0):
        # 	initTime = initTime + timedelta(days=1)
        # 	initTime = initTime.replace(hour=15, minute=40)

        # weighted_sum = 0 -> I don't reinitialize because I want to take it as a value for the next period "interpolation"
        #print("per SIMOOOOOOO")
        #print("i")
        #print(i)
        #print("lendata")
        #print(len(data))
        #print("j")
        #print(j)
        #print("lentimespan")
        #print(len(timeSpan))
        while(i>0 and timeSpan[j] > data[i]['time']):
            if not (timeSpan[j] > data[i]['time'] and timeSpan[j-1] <= data[i]['time']):
                #print("timeSpan["+str(j)+"]: "+str(timeSpan[j])+" data["+str(i)+"] : " +str(data[i]['time']) + " timeSpan["+str(j-1)+"]: "+str(timeSpan[j-1]))
                assert False
            try:
                tokens = hiv4.tokenize(data[i]['summary'])
                score = hiv4.get_score(tokens)
                #print(score)
                #subjec,10000)
                #f_score=np.multiply(score['Polarity'],sub)
                #f_score=np.multiply(f_score,0.001)
                #sentiment=np.append(sentiment,f_score)
        ##
                #summary = TextBlob(data[i]['summary']).sentiment
                polarity = score['Polarity']
                subjectivity = score['Subjectivity']
                normal_sum += polarity

                weighted_sum += polarity*subjectivity
                
                total_subj+=subjectivity
                num_sentiment +=1
            except:
                
                print("Exception")
                print(data[i]['summary'])

            i-=1
        j+=1
        weighted_sum /=total_subj
        normal_sum /=num_sentiment

        #print('line '+str(i) +"   "+ str(timeSpan[j])+"   "+ str(data[i]['time']) +"   "+ str(normal_sum))


        sentiment20minutes.append({'initTime':initTime, 'sentimentWeighted':weighted_sum, 'sentiment':normal_sum})


    #Write results
    with open('sentiment20minutesFB.csv', 'w') as csvfile:
        fieldnames = ['initTime', 'sentimentWeighted', 'sentiment']
        sentFrame = pd.DataFrame(sentiment20minutes)
        #print(sentFrame)
        sentFrame.to_csv(r'C:\Users\Utente\Desktop\NLFF\NLFF\DataSetSentiment20Delayed\sentiment20minutes'+ticker+'.csv')
        print("Finiscede with "+ticker)
       # for row in sentiment20minutes.T.iteritems():
        #	writer.writerow(row[i])

In [ ]:
print(sentiment20minutes)

In [ ]:
print(sentiment20minutes.dtype)

In [ ]:
df1 = pd.DataFrame(sentiment20minutes)
print(df1)